In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests

In [2]:
file_path = '~/Downloads/data/groups.csv'
df = pd.read_csv(file_path)

In [3]:

pis = df['pi_urls'].unique()
pi_df = pd.DataFrame(pis[1:])
pi_df.rename(columns={0: 'url'}, inplace=True)
pi_df = pi_df.append({'url': 'https://www.media.mit.edu/people/rlanger/overview/'}, ignore_index=True)
pi_df = pi_df.append({'url': 'https://www.media.mit.edu/people/dnewman/overview/'}, ignore_index=True)

/var/folders/67/4cx23sdx471d8kt2pfsw73700000gn/T/ipykernel_2350/3623007229.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pi_df = pi_df.append({'url': 'https://www.media.mit.edu/people/rlanger/overview/'}, ignore_index=True)
/var/folders/67/4cx23sdx471d8kt2pfsw73700000gn/T/ipykernel_2350/3623007229.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pi_df = pi_df.append({'url': 'https://www.media.mit.edu/people/dnewman/overview/'}, ignore_index=True)


In [4]:
def get_image_url(people_url):
    """
    scrape the media lab website to get the image corresponding to a person

    :param people_url: the url to the person's page on the media lab website
    :return: the url of the person's avatar if it exists, None otherwise
    """
    try:
        response = requests.get(people_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        hero = soup.find('div', {'class': 'hero-content-profile-avatar'})
        style = hero['style']
    except:
        return
    match = re.search('url\((".+")\)', style)
    if match:
        image_url = match.group(0)[5:-2]
        return image_url

In [5]:
def get_bio(people_url):
    try:
        response = requests.get(people_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        bio = soup.findAll('div', {'class': 'main-copy'})[0].get_text()
        return bio
    except:
        return

In [6]:
def get_name(people_url):
    try:
        response = requests.get(people_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        bio = soup.find('h1', {'class': 'hero-content-title'}).get_text()
        return bio
    except:
        return

In [7]:
pi_df['image_url'] = pi_df.apply(lambda x:get_image_url(x['url']), axis=1)

In [8]:
pi_df['bio'] = pi_df.apply(lambda x:get_bio(x['url']), axis=1)

In [9]:
pi_df['title'] = pi_df.apply(lambda x:get_name(x['url']), axis=1)

In [10]:
pi_df.set_index('title', inplace=True)

In [11]:
pi_df.to_csv('pis.csv')